In [1]:
!pip install pyTelegramBotAPI


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.0/232.0 kB 4.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyTelegramBotAPI: filename=pyTelegramBotAPI-4.12.0-py3-none-any.whl size=213956 sha256=5dcaba3e68481ba0b9e3259a4b6ff83273d8e22de4d94aadeb6208804383a325
  Stored in directory: /root/.cache/pip/wheels/7e/ba/82/f3ab5bc48525778633bccc741c0424677ed3435736221819f4
Successfully built pyTelegramBotAPI


In [2]:
# Import necessary libraries
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
import pickle


# Load dataset
url = "https://drive.google.com/uc?id=1cliBtPkNsUfc931pQQc9_cXu8y0aA22f"
data = pd.read_csv(url)

# Split into features and target
X = data.drop(columns=["target"])
y = data["target"]

# Split into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train logistic regression model
model = LogisticRegression()
model.fit(X_train, y_train)

# Evaluate model on test set
accuracy = model.score(X_test, y_test)
print("Accuracy: {:.2f}%".format(accuracy * 100))

# Save the model to a file
with open('model.pickle', 'wb') as f:
    pickle.dump(model, f)


Accuracy: 88.52%


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [3]:
import telebot
import pandas as pd
import numpy as np
import telebot
from telebot import types
from telebot import TeleBot, types
import telebot
from telebot.types import InlineKeyboardMarkup, InlineKeyboardButton

bot_token = '5600164069:AAGWOsXdsugSDSyFK2hdm4LzkrKdO1Wx3gE'
bot = telebot.TeleBot(bot_token)

In [ ]:
# Define a dictionary to store user information
user_info = {}

# Define a function to start the conversation
@bot.message_handler(commands=['start'])
def start(message):
    # Set the current step to 0 and ask for the user's age
    user_info[message.chat.id] = {}
    user_info[message.chat.id]['current_step'] = 0
    bot.send_message(message.chat.id, "Please enter your age:")


# Define a callback function to handle age input
@bot.message_handler(func=lambda message: user_info[message.chat.id]['current_step'] == 0)
def handle_age(message):
    # Store the user's age in the dictionary
    user_info[message.chat.id]['age'] = message.text
    
    # Set the current step to 1 and ask for the user's gender
    user_info[message.chat.id]['current_step'] = 1
    
    # Set up an inline keyboard with male and female options for gender
    gender_keyboard = InlineKeyboardMarkup()
    male_button = InlineKeyboardButton(text="Male", callback_data="1")
    female_button = InlineKeyboardButton(text="Female", callback_data="0")
    gender_keyboard.row(male_button, female_button)

    # Send a message to the user with the inline keyboard for gender
    bot.send_message(message.chat.id, "Please select your gender:", reply_markup=gender_keyboard)


# Define a callback function to handle gender input
@bot.callback_query_handler(func=lambda call: user_info[call.message.chat.id]['current_step'] == 1)
def handle_gender(call):
    # Store the user's gender in the dictionary
    user_info[call.message.chat.id]['gender'] = call.data
    
    # Set the current step to 2 and ask for the user's cp
    user_info[call.message.chat.id]['current_step'] = 2
    
    # Set up an inline keyboard with options for chest pain type
    cp_keyboard = InlineKeyboardMarkup()
    cp_0_button = InlineKeyboardButton(text="Typical angina", callback_data="cp_0")
    cp_1_button = InlineKeyboardButton(text="Atypical angina", callback_data="cp_1")
    cp_2_button = InlineKeyboardButton(text="Non-anginal pain", callback_data="cp_2")
    cp_3_button = InlineKeyboardButton(text="Asymptomatic", callback_data="cp_3")
    cp_keyboard.row(cp_0_button, cp_1_button)
    cp_keyboard.row(cp_2_button, cp_3_button)
    
    # Send a message to the user with the inline keyboard for chest pain type
    bot.send_message(call.message.chat.id, "Please select your chest pain type:", reply_markup=cp_keyboard)
   



# Define a callback function to handle cp input
@bot.callback_query_handler(func=lambda call: user_info[call.message.chat.id]['current_step'] == 2)
def handle_cp(call):
    # Store the user's cp in the dictionary
    user_info[call.message.chat.id]['cp'] = int(call.data[-1])
    
    # Set the current step to 3 and ask for the user's trestbps
    user_info[call.message.chat.id]['current_step'] = 3
    
    # Send a message to the user asking for their trestbps
    bot.send_message(call.message.chat.id, "Please enter your resting blood pressure (in mm Hg):")

# Define a callback function to handle trestbps input
@bot.message_handler(func=lambda message: user_info[message.chat.id]['current_step'] == 3)
def handle_trestbps(message):
    # Store the user's trestbps in the dictionary
    user_info[message.chat.id]['trestbps'] = message.text
    
    # Set the current step to 4 and ask for the user's chol
    user_info[message.chat.id]['current_step'] = 4
    bot.send_message(message.chat.id, "Please enter your serum cholesterol level (in mg/dl):")

# Define a callback function to handle chol input
@bot.message_handler(func=lambda message: user_info[message.chat.id]['current_step'] == 4)
def handle_chol(message):
    # Store the user's chol in the dictionary
    user_info[message.chat.id]['chol'] = message.text
    
    # Set the current step to 4 and ask for the user's fbs
    user_info[message.chat.id]['current_step'] = 5
    
    # Set up an inline keyboard with yes and no options for fbs
    fbs_keyboard = InlineKeyboardMarkup()
    yes_button = InlineKeyboardButton(text="Yes", callback_data="1")
    no_button = InlineKeyboardButton(text="No", callback_data="0")
    fbs_keyboard.row(yes_button, no_button)

    # Send a message to the user with the inline keyboard for fbs
    bot.send_message(message.chat.id, "Do you have a fasting blood sugar level greater than 120 mg/dl?", reply_markup=fbs_keyboard)

# Define a callback function to handle fbs input
@bot.callback_query_handler(func=lambda call: user_info[call.message.chat.id]['current_step'] == 5)
def handle_fbs(call):
    # Store the user's fbs in the dictionary
    user_info[call.message.chat.id]['fbs'] = call.data
    # Set the current step to 6 and ask for the user's restecg
    user_info[call.message.chat.id]['current_step'] = 6
    
    # Set up an inline keyboard with restecg options
    restecg_keyboard = InlineKeyboardMarkup()
    option1 = InlineKeyboardButton(text="Normal", callback_data="0")
    option2 = InlineKeyboardButton(text="ST-T wave abnormality", callback_data="1")
    option3 = InlineKeyboardButton(text="Left ventricular hypertrophy", callback_data="2")
    restecg_keyboard.row(option1)
    restecg_keyboard.row(option2)
    restecg_keyboard.row(option3)
    
    # Send a message to the user with the inline keyboard for restecg
    bot.send_message(call.message.chat.id, "What is your Resting electrocardiographic results?", reply_markup=restecg_keyboard)

# Define a callback function to handle restecg input
@bot.callback_query_handler(func=lambda call: user_info[call.message.chat.id]['current_step'] == 6)
def handle_restecg(call):
    # Store the user's restecg in the dictionary
    user_info[call.message.chat.id]['restecg'] = call.data
    # Set the current step to 5 and ask for the user's trestbps
    user_info[call.message.chat.id]['current_step'] = 7
    
    # Send a message to the user asking for their trestbps
    bot.send_message(call.message.chat.id, "Please enter your Maximum heart rate achieved (in bpm)")

# Define a callback function to handle thalach input
@bot.message_handler(func=lambda message: user_info[message.chat.id]['current_step'] == 7)
def handle_thalach(message):
    # Store the user's thalach in the dictionary
    user_info[message.chat.id]['thalach'] = message.text
    
    # Set the current step to 7 and ask for the user's exang
    user_info[message.chat.id]['current_step'] = 8
    
    # Set up an inline keyboard with yes and no options for exang
    exang_keyboard = InlineKeyboardMarkup()
    yes_button = InlineKeyboardButton(text="Yes", callback_data="1")
    no_button = InlineKeyboardButton(text="No", callback_data="0")
    exang_keyboard.row(yes_button, no_button)

    # Send a message to the user with the inline keyboard for exang
    bot.send_message(message.chat.id, "Do you have exercise-induced angina?", reply_markup=exang_keyboard)

# Define a callback function to handle exang input
@bot.callback_query_handler(func=lambda call: user_info[call.message.chat.id]['current_step'] == 8)
def handle_exang(call):
    # Store the user's exang in the dictionary
    user_info[call.message.chat.id]['exang'] = call.data
    
    # Set the current step to 8 and ask for the user's oldpeak
    user_info[call.message.chat.id]['current_step'] = 9
    bot.send_message(call.message.chat.id, "Please enter your oldpeak value:")

# Define a callback function to handle oldpeak input
@bot.message_handler(func=lambda message: user_info[message.chat.id]['current_step'] == 9)
def handle_oldpeak(message):

   # Store the user's oldpeak in the dictionary
    user_info[message.chat.id]['oldpeak'] = message.text
    
    # Set the current step to 9 and ask for the user's slope
    user_info[message.chat.id]['current_step'] = 10
    
    # Set up an inline keyboard with options for the slope of the peak exercise ST segment
    slope_keyboard = InlineKeyboardMarkup()
    upsloping_button = InlineKeyboardButton(text="Upsloping", callback_data="0")
    flat_button = InlineKeyboardButton(text="Flat", callback_data="1")
    downsloping_button = InlineKeyboardButton(text="Downsloping", callback_data="2")
    slope_keyboard.row(upsloping_button, flat_button, downsloping_button)

    # Send a message to the user with the inline keyboard for slope
    bot.send_message(message.chat.id, "Please select the slope of the peak exercise ST segment:", reply_markup=slope_keyboard)


# Define a callback function to handle slope input
@bot.callback_query_handler(func=lambda call: user_info[call.message.chat.id]['current_step'] == 10)
def handle_slope(call):
    # Store the user's slope in the dictionary
    user_info[call.message.chat.id]['slope'] = call.data
    
    # Set the current step to 11 and ask for the user's number of major vessels
    user_info[call.message.chat.id]['current_step'] = 11
    bot.send_message(call.message.chat.id, "Please enter the number of major vessels (0-3) colored by fluoroscopy:")




# Define a callback function to handle ca input
@bot.message_handler(func=lambda message: user_info[message.chat.id]['current_step'] == 11)
#@bot.callback_query_handler(func=lambda message: user_info[message.chat.id]['current_step'] == 11)
def handle_ca(message):
   
    user_info[message.chat.id]['ca'] = message.text
    

    # Set the current step to 12 and ask for the user's thal
    user_info[message.chat.id]['current_step'] = 12
    # Set up an inline keyboard with thal options
    thal_keyboard = InlineKeyboardMarkup()
    thal3_button = InlineKeyboardButton(text="3 (normal)", callback_data="3")
    thal6_button = InlineKeyboardButton(text="6 (fixed defect)", callback_data="6")
    thal7_button = InlineKeyboardButton(text="7 (reversible defect)", callback_data="7")
    thal_keyboard.row(thal3_button)
    thal_keyboard.row(thal6_button)
    thal_keyboard.row(thal7_button)
    # Send a message to the user with the inline keyboard for thal
    bot.send_message(message.chat.id, "Please select your thalassemia type:", reply_markup=thal_keyboard)
   
    
# Define a callback function to handle thal input
@bot.callback_query_handler(func=lambda call: user_info[call.message.chat.id]['current_step'] == 12)
def handle_thal(call):
  
    # Store the user's thal value in the dictionary
    user_info[call.message.chat.id]['thal'] = call.data

    # Send a message to the user with their information
    bot.send_message(call.message.chat.id, f"Your age is {user_info[call.message.chat.id]['age']}\n,your gender is {user_info[call.message.chat.id]['gender']}\n,your cp is {user_info[call.message.chat.id]['cp']}\n,your trestbps is {user_info[call.message.chat.id]['trestbps']}\n,your chol is {user_info[call.message.chat.id]['chol']}\n,your fbs is {user_info[call.message.chat.id]['fbs']}\n,your restecg is {user_info[call.message.chat.id]['restecg']}\n,your thalach is {user_info[call.message.chat.id]['thalach']}\n,your exang is {user_info[call.message.chat.id]['exang']}\n,your oldpeak is {user_info[call.message.chat.id]['oldpeak']}\n,your slope is {user_info[call.message.chat.id]['slope']}\n,your ca is {user_info[call.message.chat.id]['ca']}\n your thal is {user_info[call.message.chat.id]['thal']}.")
    


    # Print the user's information to the console
    print(user_info)

    # Retrieve the user's feature inputs from the dictionary
    age = user_info[call.message.chat.id]['age']
    gender = user_info[call.message.chat.id]['gender']
    cp = user_info[call.message.chat.id]['cp']
    trestbps = user_info[call.message.chat.id]['trestbps']
    chol = user_info[call.message.chat.id]['chol']
    fbs = user_info[call.message.chat.id]['fbs']
    restecg = user_info[call.message.chat.id]['restecg']
    thalach = user_info[call.message.chat.id]['thalach']
    exang = user_info[call.message.chat.id]['exang']
    oldpeak = user_info[call.message.chat.id]['oldpeak']
    slope = user_info[call.message.chat.id]['slope']
    ca = user_info[call.message.chat.id]['ca']
    thal = user_info[call.message.chat.id]['thal']

    # Convert the feature inputs to floats
    float_fea = [float(age), float(gender), float(cp), float(trestbps), float(chol), float(fbs), float(restecg),float(thalach), float(exang), float(oldpeak), float(slope), float(ca), float(thal)]

    # Create an array of the feature inputs
    fea = [np.array(float_fea)]

    # Load the model
    model = pickle.load(open('model.pickle', 'rb'))

    # Use the model to make a prediction
    prediction = model.predict(fea)

    # Determine the predicted status
    if prediction == 1:
        status = 'Yes'
    else:
        status = 'No'

    # Create the prediction message
    message = f"Based on the provided information, it is predicted that the patient has heart disease: {status}"

    # Send the prediction message to the user
    bot.send_message(call.message.chat.id, message)
    
    

    # Reset the user's information in the dictionary
    user_info[call.message.chat.id] = {}

    bot.send_message(message.chat.id, "Ready to make another prediction? /start")
# Start the bot
bot.polling(none_stop=True)   
